In [2]:
rawEventsRdd = sc.textFile("/home/mertergun/yahoo_data/events.txt")

In [3]:
sc.parallelize(rawEventsRdd.take(1000)).cache().saveAsTextFile("filteredEvents.txt")

In [17]:
rawEventsRdd = sc.textFile("filteredEvents.txt")
execfile("../script/context.py")
context

[<function __main__.context_city>,
 <function __main__.context_dayOfWeek>,
 <function __main__.context_time_of_day>]

In [18]:
userIdConversionDictionary = rawEventsRdd.map(get_user_id).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = rawEventsRdd.map(get_item_id).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = rawEventsRdd.map(context_city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)

In [19]:
userIdConversionDictionary

{u'001e6d8e-cbe7-4374-8c38-f37962a457e9': 1,
 u'005ca0a2-6e94-4709-adc6-f35cdd50bf8f': 0}

In [20]:
def set_context(line):
    key1 = (userIdConversionDictionary[get_user_id(line)], 
            itemIdConversionDictionary[get_item_id(line)])
    list = []
    for f in context:
        list = list + [(context.index(f), f(line))] 
    return [key1,list]
   
eventRDDConverted = rawEventsRdd.map(set_context).cache()
eventRDDConverted.take(3)

[[(1, 41), [(0, u'Tacoma'), (1, u'5'), (2, u'11')]],
 [(1, 46), [(0, u'Tacoma'), (1, u'5'), (2, u'21')]],
 [(1, 44), [(0, u'Tacoma'), (1, u'6'), (2, u'15')]]]

In [21]:
def flat_context(lst):
    l = []
    for x in lst[1]:
        l = l + [[(lst[0][0], lst[0][1], x[0]), x[1]]]
    return l
eventRDD = eventRDDConverted.flatMap(flat_context).cache()
eventRDD.take(3)

[[(1, 41, 0), u'Tacoma'], [(1, 41, 1), u'5'], [(1, 41, 2), u'11']]